In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [9]:
## Load the trained model, scaler pickle, onehot encoding
model = load_model('model.h5')

## Load encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)    

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)



In [10]:
## Example input data 

input_data = {
    'CreditScore': 600,
    'Geography': 'Spain',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [12]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography'])) 
geo_encoded_df 

d:\Anaconda\Anaconda\envs\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [14]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Spain,Male,40,3,60000,2,1,1,50000


In [19]:
## Encode the categorical data
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

ValueError: y contains previously unseen labels: 1

In [17]:
## Concatenate the encoded data
input_df = pd.concat([input_df.drop(columns=['Geography']), geo_encoded_df], axis=1)
input_df    

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,0.0,0.0,1.0


In [18]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221, -0.99850112,
        -0.57946723,  1.73494238]])

In [20]:
## Predict the value
prediction = model.predict(input_scaled)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


array([[0.02983633]], dtype=float32)

In [21]:
prediction_prob = prediction[0][0]
prediction_prob

0.029836327

In [22]:
if prediction_prob > 0.5:
    print('The customer is likely to leave the bank')
else:
    print('The customer is likely to stay with the bank')

The customer is likely to stay with the bank
